<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/06_PDF_CAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 6. PDF CAG(Credibility-Aware Generation)

## Overview  
In this exercise, we will explore how to apply Credibility-Aware Generation (CAG) techniques to process PDF documents using the Solar framework. This involves extracting text from PDFs, assessing the credibility of the content, and generating reliable outputs based on the extracted information. This notebook will guide you through the steps needed to integrate CAG with PDF handling.

## Purpose of the Exercise
The purpose of this exercise is to demonstrate the integration of Credibility-Aware Generation with PDF document processing. By the end of this tutorial, users will be able to extract text from PDFs, evaluate its credibility, and generate credible outputs using the Solar framework, enhancing the accuracy and trustworthiness of the information derived from PDF sources.


In [62]:
! pip3 install -qU langchain-upstage pypdf python-dotenv

In [63]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


![SolarSample](https://github.com/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/figures/solar_sample.png?raw=1)

In [86]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("pdfs/solar_sample.pdf")
docs = loader.load()  # or layzer.lazy_load()
print(docs[0].page_content[:1000])

Model Size Type H6 (Avg.) ARC HellaSwag MMLU TruthfulQA Winogrande GSM8KSOLAR 10.7B-Instruct⇠11BAlignment-tuned74.2071.0888.1666.2171.4383.5864.75Qwen 72B⇠72B Pretrained 73.60 65.19 85.9477.3760.19 82.4870.43Mixtral 8x7B-Instruct-v0.1⇠47B Instruction-tuned 72.62 70.22 87.63 71.16 64.58 81.37 60.73Yi 34B-200K⇠34B Pretrained 70.81 65.36 85.58 76.06 53.64 82.56 61.64Yi 34B⇠34B Pretrained 69.42 64.59 85.69 76.35 56.23 83.03 50.64Mixtral 8x7B-v0.1⇠47B Pretrained 68.42 66.04 86.49 71.82 46.78 81.93 57.47Llama 2 70B⇠70B Pretrained 67.87 67.32 87.33 69.83 44.92 83.74 54.06Falcon 180B⇠180B Pretrained 67.85 69.4588.8670.50 45.4786.9045.94SOLAR 10.7B⇠11BPretrained66.0461.9584.6065.4845.0483.6655.50Qwen 14B⇠14B Pretrained 65.86 58.28 83.99 67.70 49.43 76.80 58.98Mistral 7B-Instruct-v0.2⇠7B Instruction-tuned 65.71 63.14 84.88 60.78 68.26 77.19 40.03Yi 34B-Chat⇠34B Instruction-tuned 65.32 65.44 84.16 74.90 55.37 80.11 31.92Mistral 7B⇠7B Pretrained 60.97 59.98 83.31 64.16 42.15 78.37 37.83Table 2: Ev

In [87]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage(model="solar-pro")

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer for the question from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [88]:
chain.invoke({"question": "Explain Table 2?", "Context": docs})

'Table 2 presents evaluation results of SOLAR 10.7B and SOLAR 10.7B-Instruct along with other top-performing models. The table shows scores for six tasks and the H6 score, which is the average of the six tasks. The table also provides information about the size and training stage of the models. The best scores for H6 and the individual tasks are marked in bold. The table is taken from a document related to the Open LLM Leaderboard, which includes an explanation of the evaluation methods and the datasets used. The results in Table 2 show that SOLAR 10.7B performs well among similar-sized pretrained models, and SOLAR 10.7B-Instruct achieves the highest score in terms of H6, even outperforming larger models.'

In [92]:
chain.invoke({"question": "What is H6 score of SOLAR 10.7B-Instruct?", "Context": docs})

'The H6 score of SOLAR 10.7B-Instruct is 74.20.'

In [94]:
chain.invoke({"question": "What is ARC of Mistral?", "Context": docs})

'The information is not present in the context.'

# Excercise

How can we easily get information from complicated tables for LLMs?